In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/HEprediction")
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
import monai
from monai.data import  DataLoader, ImageDataset
from monai.transforms import (
    Resize, NormalizeIntensity, Activations, Compose, EnsureType, CenterSpatialCrop,ScaleIntensity,ResizeWithPadOrCrop, ResizeWithPadOrCropd,
    LoadImaged, EnsureChannelFirstd, EnsureTyped, NormalizeIntensityd, ScaleIntensityd,AddChannel
)
from monai.data import CacheDataset, DataLoader, ImageDataset, Dataset
from monai.data import decollate_batch
from monai.networks.nets import DenseNet121
from skimage.morphology import disk, binary_dilation, binary_erosion, remove_small_objects
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import scipy.ndimage as nd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib
import nibabel.processing
from monai.utils import set_determinism
from MyDenseNet import MyDenseNet121
import random
random.seed(123)
set_determinism(seed=123)
from monai.metrics import DiceMetric
from monai.networks.nets import SegResNet
from monai.inferers import sliding_window_inference
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
VAL_AMP = True
def inference(input):
    def _compute(input):
        return sliding_window_inference(
            inputs=input,
            roi_size=(256, 256, 32),
            sw_batch_size=4,
            predictor=model,
            overlap=0.5,
        )

    if VAL_AMP:
        with torch.cuda.amp.autocast():
            return _compute(input)
    else:
        return _compute(input)
from monai.transforms import (
    Activations, AsDiscrete,
    Compose
)
import warnings
warnings.filterwarnings('ignore')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Preprocessing for segmentation: input: CT, groundtruth; output:brain, groundtruth (512x512x48)
# Using only brain window
# Remove small objects and boundary
data_dir = "italy_ds/"
output_path = data_dir + "preprocessing"
raw_data = pd.read_csv(os.path.join(data_dir, "labels.csv"), sep=";")
for _, c_row in raw_data.iterrows():
    patient = str(c_row['filename'])
    pathCT1 = os.path.join(data_dir, "baseline", patient + ".nii.gz")
    pathCT1seg = os.path.join(data_dir, "mask", patient + "-label.nii.gz")
    # load patient and extract brain window
    print("Processing: " + patient)
    nii = nib.load(pathCT1seg)
    tmpimg1seg = np.round(nii.get_fdata()).astype(int)
    tmpimg1seg[tmpimg1seg != 1] = 0
    window_center, window_width = 40, 80
    nii = nib.load(pathCT1)
    tmpimg1 = nii.get_fdata()
    tmpimg1[tmpimg1 < 0] = 0
    tmpimg1[tmpimg1 > 200] = 0
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    tmpimg1[tmpimg1 < img_min] = img_min
    tmpimg1[tmpimg1 > img_max] = img_max
    tmpimg1 = (tmpimg1 - tmpimg1.min()) / np.ptp(tmpimg1)

    img1 = tmpimg1
    img1seg = tmpimg1seg

    img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)
    img_bw = img1.copy()
    img_bw[img_bw > 0] = 1
    for slice in range(0, img_bw.shape[2]):
        if slice <= 2 or slice > img_bw.shape[2] - 2:
            img_bw[:, :, slice] = 0
        if img_bw[:, :, slice].sum() > 0:
            img_bw[:, :, slice] = binary_erosion(img_bw[:, :, slice].astype(np.uint8),
                                                 disk(4, dtype=bool))
            img_bw[:, :, slice] = remove_small_objects(img_bw[:, :, slice].astype(bool), 1000)
            img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                  disk(4, dtype=bool))
            img_bw[:, :, slice] = nd.binary_fill_holes(img_bw[:, :, slice].astype(np.uint8))
        if img_bw[:, :, slice].sum() > 0:
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            contours = cv2.findContours(img_bw[:, :, slice].astype(np.uint8), cv2.RETR_TREE,
                                        cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] if len(contours) == 2 else contours[1]
            big_contour = max(contours, key=cv2.contourArea)
            cv2.drawContours(mask, [big_contour], -1, (255, 255, 255), -1)
            img_bw[:, :, slice][mask == 0] = 0
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            cv2.drawContours(mask, [big_contour], -1, (255, 255, 255), 20)
            img_bw[:, :, slice][mask == 255] = 0
    img1[img_bw == 0] = 0
    img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)
    resizeImage = ResizeWithPadOrCrop(spatial_size=(512, 512, 48))
    scaleImage = ScaleIntensity()
    if (img1.shape[0] != img1seg.shape[0] or img1.shape[1] != img1seg.shape[1] or img1.shape[2] != img1seg.shape[2]):
        img1_2 = np.zeros([1, img1.shape[0], img1.shape[1], img1.shape[2]])
        img1_2[0, :, :, :] = img1
        resizeSeg = ResizeWithPadOrCrop(spatial_size=(img1seg.shape[0], img1seg.shape[1], img1seg.shape[2]))
        img1_2 = resizeSeg(img1_2)
        image = np.zeros([2, img1seg.shape[0], img1seg.shape[1], img1seg.shape[2]])
        image[0:1, :, :, :] = img1_2
        image[1, :, :, :] = img1seg
        image = resizeImage(image)
    else:
        image = np.zeros([2, img1seg.shape[0], img1seg.shape[1], img1seg.shape[2]])
        image[0, :, :, :] = img1
        image[1, :, :, :] = img1seg
        image = resizeImage(image)

    empty_header = nib.Nifti1Header()
    nii = nib.load(pathCT1)
    clipped_img = nib.Nifti1Image(image[0, :, :, :], nii.affine, empty_header)
    pathCT1 = os.path.join(output_path, patient + "_brain.nii.gz")
    nib.save(clipped_img, pathCT1)

    nii = nib.load(pathCT1seg)
    dim = (nii.header["pixdim"])[1:4]
    clipped_img = nib.Nifti1Image(image[1, :, :, :], nii.affine, empty_header)
    pathCT1seg = os.path.join(output_path, patient + "_ich_seg.nii.gz")
    nib.save(clipped_img, pathCT1seg)
print("Finish")

Processing: 10028
Processing: 10042
Processing: 10052
Processing: 10056
Processing: E106342615
Finish


In [3]:
# segmentation: input: brain; output: seg_auto. Size (512x512x48)
# Using SegResNet
data_dir = "italy_ds/"
output_path=data_dir + "preprocessing"
raw_data = pd.read_csv(os.path.join(data_dir, "labels.csv"), sep=";")
xtest = []
ytest = []
nametest = []
for _, c_row in raw_data.iterrows():
    xtest = np.append(xtest, os.path.join(output_path, str(c_row['filename']) + "_brain.nii.gz"))
    ytest = np.append(ytest,
                      os.path.join(output_path, str(c_row['filename']) + "_ich_seg.nii.gz"))
    nametest = np.append(nametest, str(c_row['filename']))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_transforms = Compose(
    [
        LoadImaged(keys=["image", "groundtruth"]),
        EnsureChannelFirstd(keys=["image", "groundtruth"]),
        EnsureTyped(keys=["image", "groundtruth"]),
        NormalizeIntensityd(keys="image"),
        ScaleIntensityd(keys="image"),
    ]
)
test_files = [{"image": t2Img, "groundtruth": adcImg, "name": name} for t2Img, adcImg, name in
              zip(xtest, ytest, nametest)]
test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False,
                         num_workers=1, pin_memory=torch.cuda.is_available())
model = SegResNet(
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    init_filters=16,
    in_channels=1,
    out_channels=1,
    dropout_prob=0.2,
).to(device)
model.load_state_dict(
    torch.load(os.path.join("1_6000_new_best_metric_model_segmentation3d_1_1.pth")))

post_trans = Compose(
    [Activations(sigmoid=True), AsDiscrete(threshold=0.5)]
)

model.eval()
with torch.no_grad():
    num = 0
    for val_data in test_loader:
        val_inputs, val_labels = (
            val_data["image"].to(device),
            val_data["groundtruth"].to(device),
        )
        print(str(num) + ". Patient:" + str(val_data["name"][0]))
        num = num + 1
        val_outputs = inference(val_inputs)
        val_outputs = post_trans(val_outputs)
        t1 = val_outputs[0, 0, :, :, :].cpu().numpy()
        nii = nib.load(os.path.join(output_path, str(val_data["name"][0]) + "_" + "ich_seg.nii.gz"))
        empty_header = nib.Nifti1Header()
        affine = np.eye(4)
        affine[:3, :3] = np.diag((nii.header["pixdim"])[1:4])
        pathCT1seg = os.path.join(output_path, str(val_data["name"][0]) + "_seg_auto.nii.gz")
        imgseg = nib.Nifti1Image(t1, affine, empty_header)
        nib.save(imgseg, pathCT1seg)
print("Finish")

0. Patient:10028
1. Patient:10042
2. Patient:10052
3. Patient:10056
4. Patient:E106342615
Finish


In [4]:
# preprocessing for classification (192x192x96)
# Working from begining again
# Extract brain windows, resample to (1,1,1), skull stripping, select group of slices containing Hematoma
# Output: _brain_seg_dilation
data_dir = "italy_ds/"
output_path = data_dir + "preprocessing"
raw_data = pd.read_csv(os.path.join(data_dir, "labels.csv"), sep=";")
xtest = []
ytest = []
nametest = []
for _, c_row in raw_data.iterrows():
    patient = str(c_row['filename'])
    print("Patient:" + patient)
    pathCT1 = os.path.join(data_dir, "baseline", patient + ".nii.gz")
    pathCT1seg = os.path.join(output_path, str(c_row['filename']) + "_seg_auto.nii.gz")
    nii = nib.load(pathCT1seg)
    print("Original")
    print(nii.get_fdata().shape)
    tmp = (nii.header["pixdim"])[1:4]
    print(nii.header["pixdim"])
    image = nii.get_fdata()

    voxel_size = [1, 1, 1]
    tmpimg1seg = np.round(nii.get_fdata()).astype(int)
    tmpimg1seg[tmpimg1seg != 1] = 0

    affine = np.eye(4)
    affine[:3, :3] = np.diag((nii.header["pixdim"])[1:4])
    pathtmp = os.path.join(data_dir, "1tmp_brain_seg.nii.gz")
    empty_header = nib.Nifti1Header()
    clipped_img = nib.Nifti1Image(tmpimg1seg, affine, empty_header)
    nib.save(clipped_img, pathtmp)
    nii = nib.load(pathtmp)
    nii = nibabel.processing.resample_to_output(nii, voxel_size)
    tmpimg1seg = np.round(nii.get_fdata()).astype(int)
    tmpimg1seg[tmpimg1seg > 0] = 1

    # load bone window and resample
    window_center, window_width = 40, 80
    nii = nib.load(pathCT1)
    tmpimg1 = nii.get_fdata()
    tmpimg1[tmpimg1 < 0] = 0
    tmpimg1[tmpimg1 > 200] = 0
    resizeImage = ResizeWithPadOrCrop(spatial_size=(512, 512, 48))
    addChannel = AddChannel()
    image = resizeImage(addChannel(tmpimg1))
    tmpimg1 = image[0, :, :, :]

    pathtmp = os.path.join(data_dir, "3tmp_brain_seg.nii.gz")
    empty_header = nib.Nifti1Header()
    clipped_img = nib.Nifti1Image(tmpimg1, affine, empty_header)
    nib.save(clipped_img, pathtmp)
    nii = nib.load(pathtmp)
    nii = nibabel.processing.resample_to_output(nii, voxel_size)
    tmpimg1 = nii.get_fdata()
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    tmpimg1[tmpimg1 < img_min] = img_min
    tmpimg1[tmpimg1 > img_max] = img_max
    tmpimg1 = (tmpimg1 - tmpimg1.min()) / np.ptp(tmpimg1)
    img_bw = tmpimg1.copy()
    img_bw[img_bw > 0] = 1
    for slice in range(0, img_bw.shape[2]):
        if img_bw[:, :, slice].sum() > 0:
            img_bw[:, :, slice] = binary_erosion(img_bw[:, :, slice].astype(np.uint8),
                                                 disk(4, dtype=bool))
            img_bw[:, :, slice] = remove_small_objects(img_bw[:, :, slice].astype(bool), 1000)
            img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                  disk(4, dtype=bool))
            img_bw[:, :, slice] = nd.binary_fill_holes(img_bw[:, :, slice].astype(np.uint8))
        if img_bw[:, :, slice].sum() > 0:
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            contours = cv2.findContours(img_bw[:, :, slice].astype(np.uint8), cv2.RETR_TREE,
                                        cv2.CHAIN_APPROX_SIMPLE)
            contours = contours[0] if len(contours) == 2 else contours[1]
            big_contour = max(contours, key=cv2.contourArea)
            cv2.drawContours(mask, [big_contour], -1, (255, 255, 255), -1)
            img_bw[:, :, slice][mask == 0] = 0
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            cv2.drawContours(mask, [big_contour], -1, (255, 255, 255), 20)
            img_bw[:, :, slice][mask == 255] = 0
    tmpimg1[img_bw == 0] = 0
    tmpimg1 = 1.0 * (tmpimg1 - tmpimg1.min()) / np.ptp(tmpimg1)

    # find the first slice which contains hematoma
    for slice1 in range(tmpimg1seg.shape[2]):
        if tmpimg1seg[:, :, slice1].sum() > 0:
            break
    # find the last slice which contains hematoma
    for slice2 in range(tmpimg1seg.shape[2] - 1, 0, -1):
        if tmpimg1seg[:, :, slice2].sum() > 0:
            break
    if slice1 < slice2:
        slice1 = slice1 - 16
        if slice1 < 0:
            slice1 = 0
        slice2 = slice2 + 16
        if slice2 > tmpimg1seg.shape[2] - 1:
            slice2 = tmpimg1seg.shape[2] - 1
        if slice2 - slice1 >= 96:
            slice2 = slice1 + 96
        img1 = np.zeros([tmpimg1.shape[0], tmpimg1.shape[1], 96])
        img1seg = np.zeros([tmpimg1seg.shape[0], tmpimg1seg.shape[1], 96])
        img1[:, :, 0:slice2 - slice1] = tmpimg1[:, :, slice1:slice2]
        img1seg[:, :, 0:slice2 - slice1] = tmpimg1seg[:, :, slice1:slice2]
    else:
        img1 = np.zeros([tmpimg1.shape[0], tmpimg1.shape[1], 96])
        img1[:, :, 0:64] = tmpimg1[:, :,
                           int(np.round(tmpimg1.shape[2] / 2) - 32):int(np.round(tmpimg1.shape[2] / 2) + 32)]
        img1seg = np.zeros([tmpimg1seg.shape[0], tmpimg1seg.shape[1], 96])

    img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)

    img1dilation = img1.copy()
    img_bw = img1seg.copy()
    for slice in range(0, img_bw.shape[2]):
        if img_bw[:, :, slice].sum() > 0:
            img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                  disk(40, dtype=bool))
    img1dilation[img_bw == 0] = 0

    image = np.zeros([2, img1.shape[0], img1.shape[1], img1.shape[2]])
    image[0, :, :, :] = img1
    image[1, :, :, :] = img1dilation
    resizeImage = Resize([192, 192, 96])
    cropImage = CenterSpatialCrop(roi_size=(128, 128, 96))
    scaleImage = ScaleIntensity()
    image = resizeImage(image)
    image = cropImage(image)
    image = scaleImage(image)
    # save file
    pathCT1clipseg = os.path.join(data_dir, "classification", patient + "_brain_seg_dilation.nii.gz")
    empty_header = nib.Nifti1Header()
    nii = nib.load(pathCT1)
    clipped_img = nib.Nifti1Image(image, nii.affine, empty_header)
    nib.save(clipped_img, pathCT1clipseg)

    image = np.zeros([2, img1.shape[0], img1.shape[1], img1.shape[2]])
    image[0, :, :, :] = img1
    image[1, :, :, :] = img1seg
    image = resizeImage(image)
    image = cropImage(image)
    image = scaleImage(image)

    # save file if you want to use without dilation
    pathCT1clipseg = os.path.join(data_dir, "classification", patient + "_brain_seg.nii.gz")
    clipped_img = nib.Nifti1Image(image, nii.affine, empty_header)
    nib.save(clipped_img, pathCT1clipseg)


Patient:10028
Original
(512, 512, 48)
[1.      0.46875 0.46875 5.      1.      1.      1.      1.     ]
Patient:10042
Original
(512, 512, 48)
[1.        0.488281  0.5701952 4.5675735 1.        1.        1.
 1.       ]
Patient:10052
Original
(512, 512, 48)
[1.         0.48828125 0.48828125 3.         1.         1.
 1.         1.        ]
Patient:10056
Original
(512, 512, 48)
[1.       0.488281 0.488281 5.       1.       1.       1.       1.      ]
Patient:E106342615
Original
(512, 512, 48)
[1.         0.474609   0.47460908 3.682628   1.         1.
 1.         1.        ]


In [5]:
#Classification using MyDenseNet121 (input: 2x128x128x96)
root_dir = "italy_ds/"
data_dir = os.path.join(root_dir, "classification")
raw_data = pd.read_csv(os.path.join(root_dir, "labels.csv"), sep=";")
pathFilenames = []
labels = []
names = []
for _, c_row in raw_data.iterrows():
    pathFilenames.append(os.path.join(data_dir, str(c_row['filename']) + "_brain_seg_dilation.nii.gz"))
    labels.append(c_row['label'])
    names.append(str(c_row['filename']))
labels = np.asarray(labels).astype(int)
pathFilenames = np.asarray(pathFilenames)
names = np.asarray(names)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyDenseNet121(spatial_dims=3, in_channels=2, out_channels=128, dropout_prob=0.2).to(device)
model.cuda()
model.load_state_dict(torch.load("96dilation_test_best_metric_model_classification3d_11.pth"))
model.eval()
xval = pathFilenames
yval = labels
val_ds = ImageDataset(
    image_files=xval, labels=yval)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False,
                        num_workers=1, pin_memory=torch.cuda.is_available())
y_pred_trans = Compose([EnsureType(), Activations(sigmoid=True)])
y_trans = Compose([EnsureType()])
with torch.no_grad():
    y_pred = torch.tensor([], dtype=torch.float32, device=device)
    y = torch.tensor([], dtype=torch.double, device=device)
    for val_data in val_loader:
        inputs, val_labels = val_data[0].to(
            device), val_data[1].to(device)
        val_outputs = model(inputs)
        y_pred1 = val_outputs.flatten()
        y1 = val_labels
        y_pred = torch.cat([y_pred, y_pred1], dim=0)
        y = torch.cat([y, y1], dim=0)
    y_onehot = [y_trans(i) for i in decollate_batch(y)]
    y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
    auc_metric1 = monai.metrics.ROCAUCMetric()
    auc_metric1(y_pred_act, y_onehot)
    print("AUC total= ")
    print(auc_metric1.aggregate())
    auc_metric1.reset()


AUC total= 
0.8333333333333334
